In [2]:
'''
因为pvt_debug网络在ImageNet-1K上预训练的图片尺寸为224*224
在应用到ade20k数据集上进行分割任务时，图片尺寸变为了512*512，此时不能以224*224为基准构造位置编码，否则差别太大，网络性能下降严重
在应用到cityscapes数据集上进行分割任务时，图片尺寸变为了768*768，此时不能以224*224为基准构造位置编码，否则差别太大，网络性能下降严重
'''

import torch
import torch.nn.functional as F

size_src = (224, 224)
# size_out = (512, 512)
size_out = (768, 768)

path_src = '/home/hjf/workspace/mmsegmentation/work_dirs/pretrained_in1k/05pvt_debug/epoch_50.pth'
path_out = '/home/hjf/workspace/mmsegmentation/work_dirs/pretrained_in1k/05pvt_debug/epoch_50_img_size_{}x{}.pth'.format(*size_out)

params = torch.load(path_src, map_location='cpu')['state_dict']
# print(params.keys())

for k in params.keys():
    if 'pos_embed' in k:
        index = int(k[-1])
        pos_embed = params[k]
        # print(k, pos_embed.shape)
        H_old = size_src[0] // 2**(1+index)
        W_old = size_src[1] // 2**(1+index)
        H_new = size_out[0] // 2**(1+index)
        W_new = size_out[1] // 2**(1+index)

        # print(H_old, W_old, H_new, W_new)

        if not index == 4:
            pos_embed_new = F.interpolate(
                pos_embed.reshape(1, H_old, W_old, -1).permute(0, 3, 1, 2), 
                size=(H_new, W_new), 
                mode="bilinear"
            ).reshape(1, -1, H_new * W_new).permute(0, 2, 1)
        else:
            pos_embed_new = F.interpolate(
                pos_embed[:, 1:].reshape(1, H_old, W_old, -1).permute(0, 3, 1, 2), 
                size=(H_new, W_new), 
                mode="bilinear"
            ).reshape(1, -1, H_new * W_new).permute(0, 2, 1)
            pos_embed_new = torch.concatenate((pos_embed[:, :1], pos_embed_new), dim=1)

        print('shape of pos_embed_new: ', pos_embed_new.shape)
        params[k] = pos_embed_new
    else:
        continue

torch.save({'state_dict': params}, path_out)
print('Done!')


shape of pos_embed_new:  torch.Size([1, 36864, 64])
shape of pos_embed_new:  torch.Size([1, 9216, 128])
shape of pos_embed_new:  torch.Size([1, 2304, 256])
shape of pos_embed_new:  torch.Size([1, 577, 512])
Done!
